In [1]:
## generate Galois Fields 
有两种方法使用fpga


NameError: name '有两种方法使用fpga' is not defined

#### 方法1 使用逻辑电路实现
生成逻辑电路multi_gf.v的文件。
step1：例化 gf=GF(n)，用于计算

step2:

#### 方法2 使用rom表实现
生成rom要用的文件，这个主要就是把很大的一个逻辑电路做成查找表，但是缺点就是很可能需要把rom拼接

In [ ]:
'''
生成 乘法逻辑运算verilog 文件
'''

In [ ]:
'''
生成 加法逻辑运算verilog 文件
'''

In [ ]:
'''
生成 乘法rom表，
'''

In [ ]:
print(1)

#### 需要用到的函数和类

##### 伽罗华域的运算
生成伽罗华域运算实例，主要属性是生成多项式。

In [3]:
'''
伽罗华域运算
'''
primitive_polynomial_dict = {4: 0b10011,  # x**4  + x  + 1
                             8: (1 << 8) + 0b11101,  # x**8  + x**4  + x**3 + x**2 + 1
                             16: (1 << 16) + (1 << 12) + 0b1011,  # x**16 + x**12 + x**3 + x + 1
                             32: (1 << 32) + (1 << 22) + 0b111,  # x**32 + x**22 + x**2 + x + 1
                             64: (1 << 64) + 0b11011  # x**64 + x**4 + x**3 + x + 1
                             }
class GF:
    def __init__(self, w):
        ## gfilog  里面对应[2^0,2^1,2^2,2^3,...,...,...,none]然后对应本源多项式的码字
        ## 
        self.w = w
        self.total = (1 << self.w) - 1
        self.gflog = []
        self.gfilog = [1] # g(0) = 1
        
        self.make_gf_dict(self.w, self.gflog, self.gfilog)

    def make_gf_dict(self, w, gflog, gfilog):
        gf_element_total_number = 1<<w
        primitive_polynomial = primitive_polynomial_dict[w]#用数字表示的多项式
        for i in range(1, gf_element_total_number -1):
            temp = gfilog[i - 1] << 1  # g(i) = g(i-1) * 2
            if temp & gf_element_total_number:  # 判断溢出
                temp ^= primitive_polynomial  # 异或本原多项式
            print(i,temp)
            gfilog.append(temp)#
        #这个assert 能校验什么呀，然后为什么要加个none，代表0吗
        
        gfilog.append(None)

        for i in range(gf_element_total_number):
            gflog.append(None)

        #for i in range(0, gf_element_total_number - 1):
         #   gflog[gfilog[i]] = i
        #print(gflog)
        #print(gfilog)
        self.change_w_bit()
    def change_w_bit(self):
        w=self.w
        self.gfilog_w_bit=[]
        for i in self.gfilog:
            if(i==None):
                return 
            w_bit=bin(i)[2:]
            add_bit_number=w-len(w_bit)
            self.gfilog_w_bit.append("0"*add_bit_number+w_bit)
            

    def add(self, a, b):
        return (a ^ b) % self.total

    def sub(self, a, b):
        return (a ^ b) % self.total

    def mul(self, a, b):##这个就是先看ab对应是2^x,2^y 所以a/b=2^(x-y),a*b=2^(x+y)
        return self.gfilog[(self.gflog[a] + self.gflog[b]) % self.total]

    def div(self, a, b):
        return self.gfilog[(self.gflog[a] - self.gflog[b]) % self.total]
    def write_module_initial(self):
        ##生成模块声明和模块引用
        w=str(self.w)
        w_1=str(self.w-1)
        start="\t"
        self.write_l("module gf_mul_"+w+" "+"(")
        self.write_l(start+"input wire ["+w_1+":0] a,")
        self.write_l(start+"input wire ["+w_1+":0] b,")
        self.write_l(start+"output wire ["+w_1+":0] out")
        self.write_l(start+");")


    def write_l(self,line):
        fd=self.verilog
        level=self.level
        
        fd.write("\t"*level+line)
        fd.write("\n")
        
    def new_logic_level(self):
        self.level+=1
        
    def end_logic_level(self):
        self.level-=1

    def write_comment(self,comment_list):
        '''
        /*+++++++++++++++++++++++++++++++++++
        author_qqyk
        +comment_list_[0]
        +comment_list_[1]
        +comment_list_[2]
        +++++++++++++++++++++++++++++++++++++*/

        '''
        start="/*+++++++++++++++++++++++++++++++++++\nauthor:qqyk"
        end="+++++++++++++++++++++++++++++++++++++*/"
        start_line='++ '
        length=len(comment_list)+2
        fd=self.verilog
        for i in range(length):
            if i==0:
                fd.write(start)
            elif (i==length-1):
                fd.write(end)
            else :
                fd.write(start_line)
                fd.write(comment_list[i-1])
            fd.write("\n")
        
    def generate_module(self):
        file_name="gf_mul_"+str(self.w)
        w=self.w
        self.level=0
        with open(file_name+".v","w") as self.verilog:
            fd=self.verilog

            comment_list=[]
            #生成日期
            date=""
            comment_list.append(date)
            #描述，主要是生成矩阵。
            polynomial=""
            comment_list.append(polynomial)
            #
            #预留位置，预估需要多少门电路。
            self.write_comment(comment_list)
            ##########################
            #下面就是code了
            self.write_module_initial()
            self.write_l("")
            self.write_l("")
            self.new_logic_level()
            ############################
            #factor 就是每个系数对应的异或集合
            factor =[[] for i in range(w)]
            for i in range(w):
                for j in range(w):
                    now=i+j
                    ###查找对应系数对应的
                    factor_now=self.gfilog_w_bit[now]
                    for k in range(w):
                        if factor_now[w-1-k]=="1":
                            factor[k].append(f"(a[{i}]&&b[{j}])")
            
            for i in range(w):
                factor_of_i="^".join(factor[i])
                self.write_l(f"assign out[{i}] = {factor_of_i};")
            self.end_logic_level()
            self.write_l("endmodule")
            


            


















gf=GF(8)

        

  

1 2
2 4
3 8
4 16
5 32
6 64
7 128
8 29
9 58
10 116
11 232
12 205
13 135
14 19
15 38
16 76
17 152
18 45
19 90
20 180
21 117
22 234
23 201
24 143
25 3
26 6
27 12
28 24
29 48
30 96
31 192
32 157
33 39
34 78
35 156
36 37
37 74
38 148
39 53
40 106
41 212
42 181
43 119
44 238
45 193
46 159
47 35
48 70
49 140
50 5
51 10
52 20
53 40
54 80
55 160
56 93
57 186
58 105
59 210
60 185
61 111
62 222
63 161
64 95
65 190
66 97
67 194
68 153
69 47
70 94
71 188
72 101
73 202
74 137
75 15
76 30
77 60
78 120
79 240
80 253
81 231
82 211
83 187
84 107
85 214
86 177
87 127
88 254
89 225
90 223
91 163
92 91
93 182
94 113
95 226
96 217
97 175
98 67
99 134
100 17
101 34
102 68
103 136
104 13
105 26
106 52
107 104
108 208
109 189
110 103
111 206
112 129
113 31
114 62
115 124
116 248
117 237
118 199
119 147
120 59
121 118
122 236
123 197
124 151
125 51
126 102
127 204
128 133
129 23
130 46
131 92
132 184
133 109
134 218
135 169
136 79
137 158
138 33
139 66
140 132
141 21
142 42
143 84
144 168
145 77
146 154
147 41


生成的方式，应该是就是把每一位对应相乘出来的结果，做成一个

In [ ]:
gf.generate_module()


In [ ]:

gf1=GF(64)
gf1.generate_module()

In [5]:
print(1)



'/home/yangkun/.local/share/jupyter'